In [1]:
#Sytem
import sys,os
import datetime
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV,StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from skopt import BayesSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier




print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)

----------------------------------------
Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
scikit-learn version: 0.24.2
----------------------------------------


In [2]:
Target = "Perished"
PATH_LOG = "../logs/log.csv"
ID = 12
SEED = 42

In [3]:
df_train = pd.read_csv('../data/input/train_beta.csv')
df_test = pd.read_csv('../data/input/test_beta.csv')
sample = pd.read_csv('../data/input/gender_submission.csv')
data = pd.concat([df_train,df_test],sort=False)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)

-------------------------
Shape of Train Data
[row :891,column :22]
-------------------------
Shape of Test Data
[row :418,column :22]
-------------------------


In [4]:
drop_columns =  ["Name","Ticket","Cabin","Embarked","Title",'FareBin',"Fare_Code","Title_Code","Family","Embarked_Code"]
dataset = data.drop(columns=drop_columns)
train = dataset[:len(df_train)]
test = dataset[len(df_train):]

In [5]:
"""
各1層目モデルに対する操作（定義・訓練・予測）を簡略化するため、分類器の拡張クラスを定義します。
"""
class ClfBuilder(object):
    def __init__(self, clf, params=None):
        self.clf = clf(**params)

    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [6]:
NUM_CLASSES = 2
"""
スタッキングでは2層目のモデルに1層目のモデルの予測値を利用します。2層目で既知のデータに対する過学習を防ぐため、1層目でOut-of-Foldによる予測値を算出してこれを2層目の学習に利用します。以下の実装では、StratifiedKFoldで5分割の交差検証を行っています。
"""
def get_base_model_preds(clf, X_train, y_train, X_test):
    print(clf.clf)

    N_SPLITS = 5
    oof_valid = np.zeros((X_train.shape[0], NUM_CLASSES))
    oof_test = np.zeros((X_test.shape[0], NUM_CLASSES))
    oof_test_skf = np.zeros((N_SPLITS, X_test.shape[0], NUM_CLASSES))

    skf = StratifiedKFold(n_splits=N_SPLITS)
    for i, (train_index, valid_index) in enumerate(skf.split(X_train, y_train)):
        print('[CV] {}/{}'.format(i+1, N_SPLITS))
        X_train_, X_valid_ = X_train[train_index], X_train[valid_index]
        y_train_, y_valid_ = y_train[train_index], y_train[valid_index]

        clf.fit(X_train_, y_train_)

        oof_valid[valid_index] = clf.predict_proba(X_valid_)
        oof_test_skf[i, :] = clf.predict_proba(X_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_valid, oof_test

# 1st Layer

In [7]:
lr1_params = {

}
rfc1_1_params = {
    'bootstrap': True,
    'max_depth': 110,
    'max_features':'auto', 
    'min_samples_leaf':1,
    'min_samples_split':10, 
    'n_estimators':200,
    'random_state':SEED

}
rfc1_2_params = {
    'bootstrap': True,
    'max_depth': 10,
    'max_features':'sqrt', 
    'min_samples_leaf':1,
    'min_samples_split':5, 
    'n_estimators':1600,
    'random_state':SEED

}
gbc1_params = {
    "learning_rate":0.01,
    "max_depth":50,
    "max_features":'sqrt', 
    "min_samples_leaf":4,
    "min_samples_split":10,
    "n_estimators":200

}
etc1_params = {

}
xgbc1_1_params = {
    'n_estimators': 500, 
    'max_depth': 10,
    'random_state': 0, 
}
xgbc1_2_params = {
    'n_estimators': 500, 
    'max_depth': 100,
    'random_state': 0, 
    
}
xgbc1_3_params = {
    'n_estimators': 500, 
    'max_depth': 200,
    'random_state': 0, 
    
}
#NN
mlp1_1_params = {
    'hidden_layer_sizes':12
}
mlp1_2_params = {
    'hidden_layer_sizes':52

}
mlp1_3_params = {
    'hidden_layer_sizes':102

}
knn1_1_params = {
    'n_neighbors': 4

}
knn1_2_params = {
    'n_neighbors': 16
    
}
knn1_3_params = {
    'n_neighbors': 32
    
}

In [8]:
#Regression
lr1 = ClfBuilder(clf=LogisticRegression,params=lr1_params)
#RF
rfc1_1 = ClfBuilder(clf=RandomForestClassifier, params=rfc1_1_params)
rfc1_2 = ClfBuilder(clf=RandomForestClassifier, params=rfc1_2_params)
#Tree
gbc1 = ClfBuilder(clf=GradientBoostingClassifier, params=gbc1_params)
etc1 = ClfBuilder(clf=ExtraTreesClassifier, params=etc1_params)
xgbc1_1 = ClfBuilder(clf=XGBClassifier, params=xgbc1_1_params)
xgbc1_2 = ClfBuilder(clf=XGBClassifier, params=xgbc1_2_params)
xgbc1_3 = ClfBuilder(clf=XGBClassifier, params=xgbc1_3_params)
#NN
mlp1_1 = ClfBuilder(clf=MLPClassifier,params=mlp1_1_params)
mlp1_2 = ClfBuilder(clf=MLPClassifier,params=mlp1_2_params)
mlp1_3 = ClfBuilder(clf=MLPClassifier,params=mlp1_3_params)
#Other
knn1_1 = ClfBuilder(clf=KNeighborsClassifier,params=knn1_1_params)
knn1_2 = ClfBuilder(clf=KNeighborsClassifier,params=knn1_2_params)
knn1_3 = ClfBuilder(clf=KNeighborsClassifier,params=knn1_3_params)

In [9]:
X_train = train.iloc[:, 2:].values
y_train = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values

In [10]:
oof_valid_lr1,oof_test_lr1 = get_base_model_preds(lr1,X_train,y_train,X_test)
oof_valid_rfc1_1,oof_test_rfc1_1 = get_base_model_preds(rfc1_1,X_train,y_train,X_test)
oof_valid_rfc1_2,oof_test_rfc1_2 = get_base_model_preds(rfc1_2,X_train,y_train,X_test)
oof_valid_gbc1,oof_test_gbc1 = get_base_model_preds(gbc1,X_train,y_train,X_test)
oof_valid_etc1,oof_test_etc1 = get_base_model_preds(etc1,X_train,y_train,X_test)
oof_valid_xgbc1_1,oof_test_xgbc1_1 = get_base_model_preds(xgbc1_1,X_train,y_train,X_test)
oof_valid_xgbc1_2,oof_test_xgbc1_2 = get_base_model_preds(xgbc1_2,X_train,y_train,X_test)
oof_valid_xgbc1_3,oof_test_xgbc1_3 = get_base_model_preds(xgbc1_3,X_train,y_train,X_test)
oof_valid_mlp1_1,oof_test_mlp1_1 = get_base_model_preds(mlp1_1,X_train,y_train,X_test)
oof_valid_mlp1_2,oof_test_mlp1_2 = get_base_model_preds(mlp1_2,X_train,y_train,X_test)
oof_valid_mlp1_3,oof_test_mlp1_3 = get_base_model_preds(mlp1_3,X_train,y_train,X_test)
oof_valid_knn1_1,oof_test_knn1_1 = get_base_model_preds(knn1_1,X_train,y_train,X_test)
oof_valid_knn1_2,oof_test_knn1_2 = get_base_model_preds(knn1_2,X_train,y_train,X_test)
oof_valid_knn1_3,oof_test_knn1_3 = get_base_model_preds(knn1_3,X_train,y_train,X_test)
# oof_valid_,oof_test_ = get_base_model_preds(,X_train,y_train,X_test)

LogisticRegression()
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
RandomForestClassifier(max_depth=110, min_samples_split=10, n_estimators=200,
                       random_state=42)
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_split=5,
                       n_estimators=1600, random_state=42)
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
GradientBoostingClassifier(learning_rate=0.01, max_depth=50,
                           max_features='sqrt', min_samples_leaf=4,
                           min_samples_split=10, n_estimators=200)
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
ExtraTreesClassifier()
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=N

In [11]:
X_train_base = np.concatenate([oof_valid_lr1,
oof_valid_rfc1_1,
oof_valid_rfc1_2,
oof_valid_gbc1,
oof_valid_etc1,
oof_valid_xgbc1_1,
oof_valid_xgbc1_2,
oof_valid_xgbc1_3,
oof_valid_mlp1_1,
oof_valid_mlp1_2,
oof_valid_mlp1_3,
oof_valid_knn1_1,
oof_valid_knn1_2,
oof_valid_knn1_3
                              ], axis=1)
X_test_base = np.concatenate([
    oof_test_lr1,
    oof_test_rfc1_1,
    oof_test_rfc1_2,
    oof_test_gbc1,
    oof_test_etc1,
    oof_test_xgbc1_1,
    oof_test_xgbc1_2,
    oof_test_xgbc1_3,
    oof_test_mlp1_1,
    oof_test_mlp1_2,
    oof_test_mlp1_3,
    oof_test_knn1_1,
    oof_test_knn1_2,
    oof_test_knn1_3
                             ], axis=1)

In [12]:
xgbc2_1_params = {
    'n_estimators': 500, 
    'max_depth': 10,
    'random_state': SEED, 
}
xgbc2_2_params = {
    'n_estimators': 500, 
    'max_depth': 100,
    'random_state': SEED, 
    
}
xgbc2_3_params = {
    'n_estimators': 500, 
    'max_depth': 200,
    'random_state': SEED, 
    
}
rfc2_1_params = {
    'n_estimators': 500, 
    'max_depth': 10,
    'random_state': SEED, 
}
rfc2_2_params = {
    'n_estimators': 500, 
    'max_depth': 100,
    'random_state': SEED, 
    
}
rfc2_3_params = {
    'n_estimators': 500, 
    'max_depth': 200,
    'random_state': SEED, 
    
}

# 2nd Layer

In [13]:
xgbc2_1 = ClfBuilder(clf=XGBClassifier,params= xgbc2_1_params) 
xgbc2_2 = ClfBuilder(clf=XGBClassifier,params= xgbc2_2_params)
xgbc2_3 = ClfBuilder(clf=XGBClassifier,params=xgbc2_3_params)
rfc2_1 = ClfBuilder(clf=RandomForestClassifier,params= rfc2_1_params) 
rfc2_2 = ClfBuilder(clf=RandomForestClassifier,params= rfc2_2_params)
rfc2_3 = ClfBuilder(clf=RandomForestClassifier,params=rfc2_3_params)

In [14]:
oof_valid_2_1,oof_test_2_1 = get_base_model_preds(xgbc2_1,X_train_base,y_train,X_test_base)
oof_valid_2_2,oof_test_2_2 = get_base_model_preds(xgbc2_2,X_train_base,y_train,X_test_base)
oof_valid_2_3,oof_test_2_3 = get_base_model_preds(xgbc2_3,X_train_base,y_train,X_test_base)
oof_valid_r_1,oof_test_r_1 = get_base_model_preds(rfc2_1,X_train_base,y_train,X_test_base)
oof_valid_r_2,oof_test_r_2 = get_base_model_preds(rfc2_2,X_train_base,y_train,X_test_base)
oof_valid_r_3,oof_test_r_3 = get_base_model_preds(rfc2_3,X_train_base,y_train,X_test_base)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=42,
              reg_alpha=None, reg_lambda=None, ...)
[CV] 1/5
[CV] 2/5
[CV] 3/5
[CV] 4/5
[CV] 5/5
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorica

In [15]:
X_train_base_2 = np.concatenate([
    oof_valid_2_1,
    oof_valid_2_2,
    oof_valid_2_3,
    oof_valid_r_1,
    oof_valid_r_2,
    oof_valid_r_3
    ],axis=1
)
X_test_base_2 = np.concatenate([
    oof_test_2_1,
    oof_test_2_2,
    oof_test_2_3,
    oof_test_r_1,
    oof_test_r_2,
    oof_test_r_3
],axis=1
)

# 3rd Layer

In [22]:
final_model1 =XGBClassifier()
final_model2 = RandomForestClassifier(random_state=SEED)
final_model3 = MLPClassifier()
final_model1.fit(X_train_base_2,y_train)
final_model2.fit(X_train_base_2,y_train)
final_model3.fit(X_train_base_2,y_train)
basemodel= RandomForestClassifier(bootstrap=True,max_depth=110,max_features='auto', min_samples_leaf=1,min_samples_split=10,n_estimators=200,random_state=SEED)
basemodel.fit(X_train,y_train)

RandomForestClassifier(max_depth=110, min_samples_split=10, n_estimators=200,
                       random_state=42)

In [38]:
pred1 = final_model1.predict_proba(X_test_base_2)
pred2 = final_model2.predict_proba(X_test_base_2)
pred3 = final_model3.predict_proba(X_test_base_2)
bpred = basemodel.predict_proba(X_test)
predict = ((6*(pred1*0.5+pred2*0.3+pred3*0.2)+4*bpred)).argmax(axis=1)
predict

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,

In [37]:
STrain = final_model1.score(X_train_base_2,y_train)
Search = "x"
CV = "x"
bestparam = "x"

print('Train Score: {}'.format(STrain))

Train Score: 0.9887766554433222


In [39]:
log = pd.read_csv(PATH_LOG)
Row  =  len(log)
log.loc[Row] = "x"
log.loc[Row,"Modelname"] = "Ensemble"
log.loc[Row,"Search"] = Search
log.loc[Row,"Vestparams"] = bestparam
log.loc[Row,"CV"] = CV
log.loc[Row,"STrain"] = STrain
log.loc[Row,"SValid"] = SValid
log.loc[Row,"Age_Code_N"] = 3
use_columns = test.columns.drop(["PassengerId","Perished"])
for col in use_columns:
    log.loc[Row,col] = "o"
log.fillna("x",inplace=True)
# if PublicScore is knowed
#log.loc[Row,"SPublic"] = 0.791

In [40]:
log.to_csv(PATH_LOG,index_label=False)
log.tail()

,Modelname,STrain,SValid,Pclass,Sex,Age,SibSp,Parch,Fare,C,...,Title_Code,Embarked_Code,Search,CV,Bestparams,Title,FareBin,Fare_Code,Vestparams,Family
60,XGB,0.8918539325842697,0.8156424581005587,o,o,o,o,o,o,o,...,x,x,x,x,x,x,x,x,x,x
61,Ensemble,0.8058361391694725,0.8058361391694725,o,o,o,o,o,o,o,...,x,x,x,x,x,x,x,x,x,x
62,Ensemble,0.9932659932659933,0.9932659932659933,o,o,o,o,o,o,o,...,x,x,x,x,x,x,x,x,x,x
63,Ensemble,0.9438832772166106,0.9438832772166106,o,o,o,o,o,o,o,...,x,x,x,x,x,x,x,x,x,x
64,Ensemble,0.988777,0.826038,o,o,o,o,o,o,o,...,x,x,x,x,x,x,x,x,x,x


In [41]:
PATH_sub =  "../data/output/"+str(ID)+"_submission_"+"ensemble3"+".csv"
print(PATH_sub)
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df[Target] = predict
submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv(PATH_sub, header=True, index=False)
print("Submission Format:{} ".format(sample.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df.head(5)

../data/output/12_submission_ensemble3.csv
Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
